In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [2]:
#conda install -c conda-forge cufflinks-py
#conda install plotly
import ipywidgets as wg
from IPython.display import display

import cufflinks as cf
import chart_studio.plotly as py
import plotly.express as px

import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import inspect
import seaborn as sns

init_notebook_mode(connected=True)
cf.go_offline()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, Normalizer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_selector

pd.options.display.max_columns = 200
pd.options.display.max_rows = 272

from sklearn.feature_selection import SelectKBest, VarianceThreshold

from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, explained_variance_score, mean_absolute_error, make_scorer

from sklearn.decomposition import PCA
from joblib import dump, load

In [3]:
df = pd.read_csv("data/cleaned_df.csv", index_col="UnitID")

In [4]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
dependent_df = df[df.columns[-14:]]

In [6]:
features = df.iloc[:,:-14]

#### Running pandas.get_dummies on city of institution would lead to thousands of features and in our case would lead to a wide data set

In [7]:
features.drop(["City location of institution (HD2019)", "Institution Name"], axis=1, inplace=True)

In [8]:
features = pd.get_dummies(features, drop_first=True)

In [9]:
features.head()

,Core_Revenues,Tuition_And_Fees,Government_Grants,Private_Gifts,Investment_Return,Sales_And_Services,Other_Revenues,Tuition_And_Fees_As_Dollar_Amount,Government_Grants_As_Dollar_Amount,Private_Gifts_As_Dollar_Amount,Investment_Return_As_Dollar_Amount,Sales_And_Services_As_Dollar_Amount,Other_Revenues_As_Dollar_Amount,Core_Expenses,Instruction_Expenses,Research_Expenses,Public_Service_Expenses,Academic_Support_Expenses,Student_Service_Expenses,Institutional_Support_Expenses,Other_Core_Expenses,Instruction_Expenses_As_Dollar_Amount,Research_Expenses_As_Dollar_Amount,Public_Service_Expenses_As_Dollar_Amount,Academic_Support_Expenses_As_Dollar_Amount,Student_Service_Expenses_As_Dollar_Amount,Institutional_Support_Expenses_As_Dollar_Amount,Other_Core_Expenses_As_Dollar_Amount,Grand total instructional_staff,Grand total men instructional_staff,Grand total women instructional_staff,American Indian or Alaska Native total instructional_staff,American Indian or Alaska Native men instructional_staff,American Indian or Alaska Native women instructional_staff,Asian total instructional_staff,Asian men instructional_staff,Asian women instructional_staff,Black or African American total instructional_staff,Black or African American men instructional_staff,Black or African American women instructional_staff,Hispanic or Latino total instructional_staff,Hispanic or Latino men instructional_staff,Hispanic or Latino women instructional_staff,Native Hawaiian or Other Pacific Islander total instructional_staff,Native Hawaiian or Other Pacific Islander men instructional_staff,Native Hawaiian or Other Pacific Islander women instructional_staff,White total instructional_staff,White men instructional_staff,White women instructional_staff,Two or more races total instructional_staff,Two or more races men instructional_staff,Two or more races women instructional_staff,Race/ethnicity unknown total instructional_staff,Race/ethnicity unknown men instructional_staff,Race/ethnicity unknown women instructional_staff,Nonresident alien total instructional_staff,Nonresident alien men instructional_staff,Nonresident alien women instructional_staff,Grand total men instructional_staff_as_percentage,Grand total women instructional_staff_as_percentage,American Indian or Alaska Native total instructional_staff_as_percentage,American Indian or Alaska Native men instructional_staff_as_percentage,American Indian or Alaska Native women instructional_staff_as_percentage,Asian total instructional_staff_as_percentage,Asian men instructional_staff_as_percentage,Asian women instructional_staff_as_percentage,Black or African American total instructional_staff_as_percentage,Black or African American men instructional_staff_as_percentage,Black or African American women instructional_staff_as_percentage,Hispanic or Latino total instructional_staff_as_percentage,Hispanic or Latino men instructional_staff_as_percentage,Hispanic or Latino women instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander total instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander men instructional_staff_as_percentage,Native Hawaiian or Other Pacific Islander women instructional_staff_as_percentage,White total instructional_staff_as_percentage,White men instructional_staff_as_percentage,White women instructional_staff_as_percentage,Two or more races total instructional_staff_as_percentage,Two or more races men instructional_staff_as_percentage,Two or more races women instructional_staff_as_percentage,Race/ethnicity unknown total instructional_staff_as_percentage,Race/ethnicity unknown men instructional_staff_as_percentage,Race/ethnicity unknown women instructional_staff_as_percentage,Nonresident alien total instructional_staff_as_percentage,Nonresident alien men instructional_staff_as_percentage,Nonresident alien women instructional_staff_as_percentage,Percent of full-time first-time undergraduates awarded any financial aid (SFA1819),Percent of full-time first-time und

In [10]:
dependent_df.head()

,Graduation rate total cohort (DRVGR2019),Graduation rate men (DRVGR2019),Graduation rate women (DRVGR2019),Graduation rate American Indian or Alaska Native (DRVGR2019),Graduation rate Asian/Native Hawaiian/Other Pacific Islander (DRVGR2019),Graduation rate Asian (DRVGR2019),Graduation rate Native Hawaiian or Other Pacific Islander (DRVGR2019),Graduation rate Black non-Hispanic (DRVGR2019),Graduation rate Hispanic (DRVGR2019),Graduation rate White non-Hispanic (DRVGR2019),Graduation rate two or more races (DRVGR2019),Graduation rate Race/ethnicity unknown (DRVGR2019),Graduation rate Nonresident alien (DRVGR2019),Transfer-out rate total cohort (DRVGR2019)
UnitID,,,,,,,,,,,,,,
180203,29.0,13.0,46.0,26.0,NaN,NaN,NaN,NaN,NaN,33.0,NaN,NaN,NaN,NaN
222178,61.0,56.0,65.0,25.0,58.0,58.0,NaN,53.0,50.0,67.0,54.0,0.0,57.0,NaN
138558,26.0,28.0,25.0,33.0,25.0,25.0,NaN,12.0,32.0,29.0,20.0,100.0,33.0,39.0
172866,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN
108232,45.0,38.0,51.0,67.0,56.0,57.0,40.0,20.0,33.0,50.0,40.0,28.0,57.0,14.0


In [11]:
df.drop(["City location of institution (HD2019)", "Institution Name"], axis=1, inplace=True)

## train model and return results

In [12]:
def train_model(model, X, y):
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    return (mae, mse)


#### Dictionary holds paramter key value pairs along with model results recorded for each experiment

In [13]:
baseline_epoch = []

In [14]:
num_cols = list(range(124)) #numeric column numbers

In [15]:
X = features
y = df.iloc[:,-14]

# Ridge Regression Pipeline

In [16]:
model = Pipeline(
    [   #Impute Numeric Columns
        ("Imputer", ColumnTransformer([
            ("Impute", SimpleImputer(), num_cols)
        ], remainder='passthrough')),
        
        #Scale Numeric Columns
        ("Scaler", ColumnTransformer([
            ("Scale", StandardScaler(), num_cols)
        ], remainder='passthrough')),
        
        ("classifier", ElasticNet(alpha=0.1, l1_ratio=0.5))
    ]
)

### First predicting total graduation rates

In [17]:
mae_total, mse_total = train_model(model, X,y)

### predicting African American Graduation Rates

In [18]:
# first drop where African American Graduation Rates are not available
df_black = df.dropna(subset=[df.columns[-7]])

In [19]:
X_black = df_black.iloc[:,:-14]
y_black = df_black.iloc[:,-7]

In [20]:
X_black = pd.get_dummies(X_black, drop_first=True)

In [21]:
mae_black, mse_black = train_model(model, X_black, y_black)

In [22]:
mae_black, mse_black

(14.861809376780581, 411.01293077069744)

In [23]:
results = {
    "model": model,
    "alpha": 0.1,
    "l1_ratio": 0.5,
    "mae_total": mae_total,
    "mse_total": mse_total,
    "mae_black": mae_black,
    "mse_black": mse_black
}

In [24]:
baseline_epoch.append(results)

In [25]:
dump(baseline_epoch, "data/model_logging.joblib")

['data/model_logging.joblib']

In [26]:
baseline_epoch[0]

{'model': Pipeline(steps=[('Imputer',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('Impute', SimpleImputer(),
                                                   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                    10, 11, 12, 13, 14, 15, 16,
                                                    17, 18, 19, 20, 21, 22, 23,
                                                    24, 25, 26, 27, 28, 29, ...])])),
                 ('Scaler',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('Scale', StandardScaler(),
                                                   [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                    10, 11, 12, 13, 14, 15, 16,
                                                    17, 18, 19, 20, 21, 22, 23,
                                                    24, 25, 26, 27, 28, 29, ...])])),
